# Advanced Banking Analytics with OLAP Operations

**Objective:** Demonstrate advanced analytics scenarios with direct OpenSearch queries and OLAP-style operations.

**What You'll Learn:**
1. Direct OpenSearch API calls (no abstraction layers)
2. OLAP operations (slice, dice, drill-down, roll-up, pivot)
3. Complex multi-dimensional analysis
4. Real-time aggregations and analytics
5. Advanced vector search with filters

**OLAP Implementation:**
- **Slice:** Filter data on one dimension
- **Dice:** Filter on multiple dimensions
- **Drill-Down:** Navigate from summary to detail
- **Roll-Up:** Aggregate from detail to summary
- **Pivot:** Rotate data for different views

## 1. Setup - Direct OpenSearch Connection

In [ ]:
# Standard notebook setup
import sys
from pathlib import Path

# Add project root to path
project_root = Path.cwd().parent.parent
if str(project_root) not in sys.path:
    sys.path.insert(0, str(project_root))

# Apply nest_asyncio for Jupyter compatibility
import nest_asyncio
nest_asyncio.apply()

# Core imports
import pandas as pd
import numpy as np
from datetime import datetime, timedelta
import json
from opensearchpy import OpenSearch
import warnings
warnings.filterwarnings('ignore')

# Direct OpenSearch connection using config
OPENSEARCH_HOST = 'localhost'
OPENSEARCH_PORT = 9200

opensearch_client = OpenSearch(
    hosts=[{'host': OPENSEARCH_HOST, 'port': OPENSEARCH_PORT}],
    http_auth=None,
    use_ssl=False,
    verify_certs=False
)

print("✅ Setup complete")
print(f"   Project root: {project_root}")
try:
    info = opensearch_client.info()
    print(f"   OpenSearch Cluster: {info['cluster_name']}")
    print(f"   OpenSearch Version: {info['version']['number']}")
except Exception as e:
    print(f"⚠️  OpenSearch not available: {e}")

## 2. Complex Scenario 1: Multi-Jurisdictional Sanctions Network

**Scenario:** A sophisticated money laundering network operates across multiple jurisdictions using:
- Shell companies with similar names
- Multiple aliases and transliterations
- Coordinated transactions across time zones
- Layered ownership structures

In [ ]:
# Direct OpenSearch Query: Complex Boolean Search with Vector Similarity
complex_query = {
    "size": 20,
    "query": {
        "bool": {
            "should": [
                {
                    "knn": {
                        "embedding": {
                            "vector": [0.1] * 384,  # Placeholder - would use actual embedding
                            "k": 10
                        }
                    }
                },
                {
                    "multi_match": {
                        "query": "John Doe International Trading",
                        "fields": ["name^3", "aliases^2", "counterparty"],
                        "fuzziness": "AUTO"
                    }
                }
            ],
            "filter": [
                {
                    "terms": {
                        "list_type": ["OFAC", "EU_SANCTIONS", "UN_SANCTIONS"]
                    }
                },
                {
                    "range": {
                        "added_date": {
                            "gte": "2020-01-01"
                        }
                    }
                }
            ],
            "minimum_should_match": 1
        }
    },
    "_source": ["name", "aliases", "country", "list_type", "added_date"]
}

print("🔍 Complex Multi-Criteria Search Query:")
print(json.dumps(complex_query, indent=2))

# Execute query
try:
    response = opensearch_client.search(
        index="sanctions_list",
        body=complex_query
    )
    
    print(f"\n✅ Query executed successfully")
    print(f"   Total hits: {response['hits']['total']['value']}")
    print(f"   Max score: {response['hits']['max_score']}")
    
    for hit in response['hits']['hits']:
        print(f"\n   Match: {hit['_source']['name']}")
        print(f"   Score: {hit['_score']:.4f}")
        print(f"   List: {hit['_source']['list_type']}")
except Exception as e:
    print(f"⚠️  Query error: {e}")

## 3. OLAP Operation: SLICE

**Definition:** Extract a subset of data by fixing one dimension.

**Example:** Slice transactions by specific time period (Q1 2024)

In [ ]:
# OLAP SLICE: Filter by time dimension
slice_query = {
    "size": 0,  # Only aggregations, no documents
    "query": {
        "range": {
            "timestamp": {
                "gte": "2024-01-01",
                "lt": "2024-04-01"
            }
        }
    },
    "aggs": {
        "total_volume": {
            "sum": {"field": "amount"}
        },
        "transaction_count": {
            "value_count": {"field": "transaction_id"}
        },
        "avg_amount": {
            "avg": {"field": "amount"}
        },
        "by_type": {
            "terms": {
                "field": "transaction_type",
                "size": 10
            },
            "aggs": {
                "type_volume": {
                    "sum": {"field": "amount"}
                }
            }
        }
    }
}

print("📊 OLAP SLICE Operation: Q1 2024 Transactions")
print("="*60)

try:
    response = opensearch_client.search(
        index="aml_transactions",
        body=slice_query
    )
    
    aggs = response['aggregations']
    print(f"\n✅ Slice Results (Q1 2024):")
    print(f"   Total Volume: ${aggs['total_volume']['value']:,.2f}")
    print(f"   Transaction Count: {aggs['transaction_count']['value']:,}")
    print(f"   Average Amount: ${aggs['avg_amount']['value']:,.2f}")
    
    print(f"\n   By Transaction Type:")
    for bucket in aggs['by_type']['buckets']:
        print(f"     {bucket['key']:15s}: {bucket['doc_count']:4d} txns (${bucket['type_volume']['value']:,.2f})")
        
except Exception as e:
    print(f"⚠️  Query error: {e}")

## 4. OLAP Operation: DICE

**Definition:** Filter on multiple dimensions simultaneously.

**Example:** High-value international wire transfers in suspicious countries

In [ ]:
# OLAP DICE: Multi-dimensional filter
dice_query = {
    "size": 0,
    "query": {
        "bool": {
            "filter": [
                {"term": {"transaction_type": "WIRE_TRANSFER"}},
                {"range": {"amount": {"gte": 10000}}},
                {"terms": {"currency": ["USD", "EUR", "GBP"]}},
                {
                    "range": {
                        "timestamp": {
                            "gte": "now-30d/d"
                        }
                    }
                }
            ]
        }
    },
    "aggs": {
        "by_account": {
            "terms": {
                "field": "account_id",
                "size": 20,
                "order": {"total_amount": "desc"}
            },
            "aggs": {
                "total_amount": {
                    "sum": {"field": "amount"}
                },
                "transaction_count": {
                    "value_count": {"field": "transaction_id"}
                },
                "unique_counterparties": {
                    "cardinality": {"field": "counterparty"}
                }
            }
        },
        "time_distribution": {
            "date_histogram": {
                "field": "timestamp",
                "calendar_interval": "day"
            },
            "aggs": {
                "daily_volume": {
                    "sum": {"field": "amount"}
                }
            }
        }
    }
}

print("📊 OLAP DICE Operation: High-Value International Wires")
print("="*60)
print("Filters:")
print("  - Type: WIRE_TRANSFER")
print("  - Amount: >= $10,000")
print("  - Currency: USD, EUR, GBP")
print("  - Period: Last 30 days")

try:
    response = opensearch_client.search(
        index="aml_transactions",
        body=dice_query
    )
    
    print(f"\n✅ Dice Results:")
    print(f"   Total Matching: {response['hits']['total']['value']:,}")
    
    print(f"\n   Top 10 Accounts by Volume:")
    for bucket in response['aggregations']['by_account']['buckets'][:10]:
        print(f"     {bucket['key']:15s}: ${bucket['total_amount']['value']:>12,.2f} ({bucket['transaction_count']['value']:3d} txns, {bucket['unique_counterparties']['value']:2d} counterparties)")
        
except Exception as e:
    print(f"⚠️  Query error: {e}")

## 5. OLAP Operation: DRILL-DOWN

**Definition:** Navigate from summary to detailed data.

**Example:** Start with country-level, drill to account-level, then transaction-level

In [ ]:
# OLAP DRILL-DOWN: Hierarchical aggregation
drilldown_query = {
    "size": 0,
    "aggs": {
        "level1_currency": {
            "terms": {
                "field": "currency",
                "size": 10
            },
            "aggs": {
                "currency_volume": {
                    "sum": {"field": "amount"}
                },
                "level2_type": {
                    "terms": {
                        "field": "transaction_type",
                        "size": 5
                    },
                    "aggs": {
                        "type_volume": {
                            "sum": {"field": "amount"}
                        },
                        "level3_account": {
                            "terms": {
                                "field": "account_id",
                                "size": 3
                            },
                            "aggs": {
                                "account_volume": {
                                    "sum": {"field": "amount"}
                                },
                                "stats": {
                                    "stats": {"field": "amount"}
                                }
                            }
                        }
                    }
                }
            }
        }
    }
}

print("📊 OLAP DRILL-DOWN: Currency → Type → Account")
print("="*60)

try:
    response = opensearch_client.search(
        index="aml_transactions",
        body=drilldown_query
    )
    
    print("\n✅ Hierarchical Drill-Down Results:\n")
    
    for currency in response['aggregations']['level1_currency']['buckets'][:3]:
        print(f"📍 Level 1 - Currency: {currency['key']}")
        print(f"   Volume: ${currency['currency_volume']['value']:,.2f}")
        print(f"   Transactions: {currency['doc_count']:,}")
        
        for txn_type in currency['level2_type']['buckets'][:2]:
            print(f"\n   📍 Level 2 - Type: {txn_type['key']}")
            print(f"      Volume: ${txn_type['type_volume']['value']:,.2f}")
            print(f"      Transactions: {txn_type['doc_count']:,}")
            
            for account in txn_type['level3_account']['buckets'][:2]:
                print(f"\n      📍 Level 3 - Account: {account['key']}")
                print(f"         Volume: ${account['account_volume']['value']:,.2f}")
                print(f"         Avg: ${account['stats']['avg']:,.2f}")
                print(f"         Min: ${account['stats']['min']:,.2f}")
                print(f"         Max: ${account['stats']['max']:,.2f}")
        print("\n" + "-"*60)
        
except Exception as e:
    print(f"⚠️  Query error: {e}")

## 6. OLAP Operation: ROLL-UP

**Definition:** Aggregate from detailed to summary level.

**Example:** Roll up daily transactions to weekly, then monthly summaries

In [ ]:
# OLAP ROLL-UP: Time-based aggregation at multiple levels
rollup_query = {
    "size": 0,
    "aggs": {
        "daily": {
            "date_histogram": {
                "field": "timestamp",
                "calendar_interval": "day"
            },
            "aggs": {
                "daily_volume": {"sum": {"field": "amount"}},
                "daily_count": {"value_count": {"field": "transaction_id"}}
            }
        },
        "weekly": {
            "date_histogram": {
                "field": "timestamp",
                "calendar_interval": "week"
            },
            "aggs": {
                "weekly_volume": {"sum": {"field": "amount"}},
                "weekly_count": {"value_count": {"field": "transaction_id"}},
                "weekly_avg": {"avg": {"field": "amount"}}
            }
        },
        "monthly": {
            "date_histogram": {
                "field": "timestamp",
                "calendar_interval": "month"
            },
            "aggs": {
                "monthly_volume": {"sum": {"field": "amount"}},
                "monthly_count": {"value_count": {"field": "transaction_id"}},
                "monthly_avg": {"avg": {"field": "amount"}},
                "monthly_max": {"max": {"field": "amount"}}
            }
        }
    }
}

print("📊 OLAP ROLL-UP: Daily → Weekly → Monthly")
print("="*60)

try:
    response = opensearch_client.search(
        index="aml_transactions",
        body=rollup_query
    )
    
    print("\n✅ Roll-Up Results:\n")
    
    # Monthly summary
    print("📅 Monthly Summary:")
    for bucket in response['aggregations']['monthly']['buckets'][:6]:
        date = bucket['key_as_string'][:7]  # YYYY-MM
        print(f"   {date}: ${bucket['monthly_volume']['value']:>12,.2f} ({bucket['monthly_count']['value']:>4,} txns, avg: ${bucket['monthly_avg']['value']:>8,.2f})")
    
    # Weekly summary
    print("\n📅 Weekly Summary (last 4 weeks):")
    for bucket in response['aggregations']['weekly']['buckets'][-4:]:
        date = bucket['key_as_string'][:10]  # YYYY-MM-DD
        print(f"   Week of {date}: ${bucket['weekly_volume']['value']:>12,.2f} ({bucket['weekly_count']['value']:>4,} txns)")
    
    # Daily summary
    print("\n📅 Daily Summary (last 7 days):")
    for bucket in response['aggregations']['daily']['buckets'][-7:]:
        date = bucket['key_as_string'][:10]
        print(f"   {date}: ${bucket['daily_volume']['value']:>12,.2f} ({bucket['daily_count']['value']:>4,} txns)")
        
except Exception as e:
    print(f"⚠️  Query error: {e}")

## 7. OLAP Operation: PIVOT

**Definition:** Rotate data to view from different perspectives.

**Example:** Create a pivot table of Transaction Type vs Currency

In [ ]:
# OLAP PIVOT: Multi-dimensional matrix
pivot_query = {
    "size": 0,
    "aggs": {
        "transaction_types": {
            "terms": {
                "field": "transaction_type",
                "size": 10
            },
            "aggs": {
                "by_currency": {
                    "terms": {
                        "field": "currency",
                        "size": 10
                    },
                    "aggs": {
                        "volume": {"sum": {"field": "amount"}},
                        "count": {"value_count": {"field": "transaction_id"}},
                        "avg_amount": {"avg": {"field": "amount"}}
                    }
                }
            }
        }
    }
}

print("📊 OLAP PIVOT: Transaction Type × Currency Matrix")
print("="*60)

try:
    response = opensearch_client.search(
        index="aml_transactions",
        body=pivot_query
    )
    
    # Build pivot table
    pivot_data = []
    for txn_type in response['aggregations']['transaction_types']['buckets']:
        for currency in txn_type['by_currency']['buckets']:
            pivot_data.append({
                'Type': txn_type['key'],
                'Currency': currency['key'],
                'Volume': currency['volume']['value'],
                'Count': currency['count']['value'],
                'Avg': currency['avg_amount']['value']
            })
    
    df = pd.DataFrame(pivot_data)
    
    # Create pivot table
    pivot_volume = df.pivot_table(
        values='Volume',
        index='Type',
        columns='Currency',
        aggfunc='sum',
        fill_value=0
    )
    
    pivot_count = df.pivot_table(
        values='Count',
        index='Type',
        columns='Currency',
        aggfunc='sum',
        fill_value=0
    )
    
    print("\n✅ Pivot Table - Transaction Volume by Type and Currency:\n")
    print(pivot_volume.to_string())
    
    print("\n\n✅ Pivot Table - Transaction Count by Type and Currency:\n")
    print(pivot_count.to_string())
    
    # Add totals
    print("\n\n📊 Row Totals (by Type):")
    for idx in pivot_volume.index:
        print(f"   {idx:20s}: ${pivot_volume.loc[idx].sum():>15,.2f}")
    
    print("\n📊 Column Totals (by Currency):")
    for col in pivot_volume.columns:
        print(f"   {col:10s}: ${pivot_volume[col].sum():>15,.2f}")
        
except Exception as e:
    print(f"⚠️  Query error: {e}")

## 8. Complex Scenario 2: Real-Time Fraud Ring Detection

**Scenario:** Detect coordinated fraud rings using:
- Shared device fingerprints
- Similar transaction patterns
- Temporal correlation
- Geographic clustering

In [ ]:
# Complex fraud ring detection query
fraud_ring_query = {
    "size": 0,
    "query": {
        "bool": {
            "filter": [
                {"range": {"timestamp": {"gte": "now-24h"}}},
                {"range": {"amount": {"gte": 1000, "lte": 9999}}}
            ]
        }
    },
    "aggs": {
        "by_counterparty": {
            "terms": {
                "field": "counterparty",
                "size": 100,
                "min_doc_count": 3
            },
            "aggs": {
                "unique_accounts": {
                    "cardinality": {"field": "account_id"}
                },
                "total_volume": {
                    "sum": {"field": "amount"}
                },
                "time_spread": {
                    "stats": {"field": "timestamp"}
                },
                "amount_stats": {
                    "stats": {"field": "amount"}
                },
                "amount_variance": {
                    "extended_stats": {"field": "amount"}
                },
                "accounts": {
                    "terms": {
                        "field": "account_id",
                        "size": 20
                    }
                }
            }
        }
    }
}

print("🚨 Complex Scenario: Fraud Ring Detection")
print("="*60)
print("Detection Criteria:")
print("  - Multiple accounts (>3) transacting with same counterparty")
print("  - Similar amounts ($1,000-$9,999)")
print("  - Within 24-hour window")
print("  - Low amount variance (coordinated)")

try:
    response = opensearch_client.search(
        index="aml_transactions",
        body=fraud_ring_query
    )
    
    print("\n✅ Potential Fraud Rings Detected:\n")
    
    suspicious_rings = []
    for bucket in response['aggregations']['by_counterparty']['buckets']:
        unique_accounts = bucket['unique_accounts']['value']
        variance = bucket['amount_variance']['std_deviation']
        
        # Flag if multiple accounts with low variance (coordinated)
        if unique_accounts >= 3 and variance < 500:
            suspicious_rings.append({
                'counterparty': bucket['key'],
                'accounts': unique_accounts,
                'transactions': bucket['doc_count'],
                'volume': bucket['total_volume']['value'],
                'avg_amount': bucket['amount_stats']['avg'],
                'variance': variance
            })
    
    if suspicious_rings:
        for i, ring in enumerate(sorted(suspicious_rings, key=lambda x: x['accounts'], reverse=True), 1):
            print(f"🚨 Ring #{i}: {ring['counterparty']}")
            print(f"   Accounts Involved: {ring['accounts']}")
            print(f"   Transactions: {ring['transactions']}")
            print(f"   Total Volume: ${ring['volume']:,.2f}")
            print(f"   Avg Amount: ${ring['avg_amount']:,.2f}")
            print(f"   Amount Variance: ${ring['variance']:.2f} (LOW = Coordinated)")
            print(f"   Risk Score: {min(100, ring['accounts'] * 10 + (1000 - ring['variance']) / 10):.0f}/100")
            print()
    else:
        print("   No suspicious fraud rings detected in last 24 hours")
        
except Exception as e:
    print(f"⚠️  Query error: {e}")

## 9. Complex Scenario 3: Cross-Border Money Laundering Network

**Scenario:** Detect layered money laundering across jurisdictions:
- Placement: Initial deposits in multiple accounts
- Layering: Complex web of transfers
- Integration: Final withdrawal to legitimate business

In [ ]:
# Multi-stage money laundering detection
ml_network_query = {
    "size": 0,
    "aggs": {
        "placement_stage": {
            "filter": {
                "bool": {
                    "must": [
                        {"term": {"transaction_type": "DEPOSIT"}},
                        {"range": {"amount": {"gte": 5000, "lte": 9999}}}
                    ]
                }
            },
            "aggs": {
                "accounts": {
                    "terms": {"field": "account_id", "size": 50},
                    "aggs": {
                        "deposit_count": {"value_count": {"field": "transaction_id"}},
                        "total_deposits": {"sum": {"field": "amount"}}
                    }
                }
            }
        },
        "layering_stage": {
            "filter": {
                "term": {"transaction_type": "TRANSFER"}
            },
            "aggs": {
                "transfer_network": {
                    "terms": {"field": "account_id", "size": 50},
                    "aggs": {
                        "transfer_count": {"value_count": {"field": "transaction_id"}},
                        "unique_counterparties": {"cardinality": {"field": "counterparty"}},
                        "total_transferred": {"sum": {"field": "amount"}}
                    }
                }
            }
        },
        "integration_stage": {
            "filter": {
                "bool": {
                    "must": [
                        {"term": {"transaction_type": "WITHDRAWAL"}},
                        {"range": {"amount": {"gte": 10000}}}
                    ]
                }
            },
            "aggs": {
                "large_withdrawals": {
                    "terms": {"field": "account_id", "size": 50},
                    "aggs": {
                        "withdrawal_total": {"sum": {"field": "amount"}}
                    }
                }
            }
        }
    }
}

print("🌍 Complex Scenario: Cross-Border Money Laundering Network")
print("="*60)
print("Three-Stage Detection:")
print("  1. PLACEMENT: Multiple structured deposits")
print("  2. LAYERING: Complex transfer network")
print("  3. INTEGRATION: Large withdrawals")

try:
    response = opensearch_client.search(
        index="aml_transactions",
        body=ml_network_query
    )
    
    aggs = response['aggregations']
    
    print("\n✅ Money Laundering Network Analysis:\n")
    
    # Stage 1: Placement
    print("📍 Stage 1 - PLACEMENT (Structured Deposits):")
    placement_accounts = aggs['placement_stage']['accounts']['buckets'][:5]
    for acc in placement_accounts:
        if acc['deposit_count']['value'] >= 3:
            print(f"   ⚠️  {acc['key']}: {acc['deposit_count']['value']} deposits = ${acc['total_deposits']['value']:,.2f}")
    
    # Stage 2: Layering
    print("\n📍 Stage 2 - LAYERING (Transfer Network):")
    layering_accounts = aggs['layering_stage']['transfer_network']['buckets'][:5]
    for acc in layering_accounts:
        if acc['unique_counterparties']['value'] >= 5:
            print(f"   ⚠️  {acc['key']}: {acc['transfer_count']['value']} transfers to {acc['unique_counterparties']['value']} parties = ${acc['total_transferred']['value']:,.2f}")
    
    # Stage 3: Integration
    print("\n📍 Stage 3 - INTEGRATION (Large Withdrawals):")
    integration_accounts = aggs['integration_stage']['large_withdrawals']['buckets'][:5]
    for acc in integration_accounts:
        print(f"   ⚠️  {acc['key']}: ${acc['withdrawal_total']['value']:,.2f} withdrawn")
    
    # Find accounts in all 3 stages
    placement_ids = {a['key'] for a in placement_accounts}
    layering_ids = {a['key'] for a in layering_accounts}
    integration_ids = {a['key'] for a in integration_accounts}
    
    complete_cycle = placement_ids & layering_ids & integration_ids
    
    if complete_cycle:
        print("\n🚨 HIGH RISK: Accounts in ALL 3 stages (complete laundering cycle):")
        for acc_id in complete_cycle:
            print(f"   🔴 {acc_id} - IMMEDIATE INVESTIGATION REQUIRED")
    else:
        print("\n✅ No accounts detected in all 3 stages")
        
except Exception as e:
    print(f"⚠️  Query error: {e}")

## 10. Advanced Vector Search with Business Logic

**Scenario:** Combine vector similarity with business rules for intelligent screening

In [ ]:
# Advanced hybrid search: Vector + Business Rules
hybrid_search_query = {
    "size": 10,
    "query": {
        "script_score": {
            "query": {
                "bool": {
                    "should": [
                        {
                            "knn": {
                                "embedding": {
                                    "vector": [0.1] * 384,  # Replace with actual embedding
                                    "k": 20
                                }
                            }
                        }
                    ],
                    "filter": [
                        {"terms": {"list_type": ["OFAC", "EU_SANCTIONS"]}}
                    ]
                }
            },
            "script": {
                "source": """
                    double vector_score = _score;
                    double business_multiplier = 1.0;
                    
                    // Apply business rules
                    if (doc['risk_level'].value == 'high') {
                        business_multiplier = 2.0;
                    } else if (doc['risk_level'].value == 'medium') {
                        business_multiplier = 1.5;
                    }
                    
                    // Boost recent additions
                    if (doc['added_date'].size() > 0) {
                        long days_old = (System.currentTimeMillis() - doc['added_date'].value.toInstant().toEpochMilli()) / (1000*60*60*24);
                        if (days_old < 90) {
                            business_multiplier *= 1.2;
                        }
                    }
                    
                    return vector_score * business_multiplier;
                """
            }
        }
    },
    "_source": ["name", "list_type", "risk_level", "added_date", "country"]
}

print("🔍 Advanced Hybrid Search: Vector + Business Rules")
print("="*60)
print("Combines:")
print("  - Vector similarity (k-NN)")
print("  - Risk level boosting")
print("  - Recency boosting")

try:
    response = opensearch_client.search(
        index="sanctions_list",
        body=hybrid_search_query
    )
    
    print(f"\n✅ Hybrid Search Results:")
    print(f"   Total Hits: {response['hits']['total']['value']}")
    
    for i, hit in enumerate(response['hits']['hits'][:5], 1):
        source = hit['_source']
        print(f"\n   {i}. {source.get('name', 'Unknown')}")
        print(f"      List: {source.get('list_type', 'N/A')}")
        print(f"      Risk: {source.get('risk_level', 'N/A')}")
        print(f"      Score: {hit['_score']:.4f}")
        
except Exception as e:
    print(f"⚠️  Query error: {e}")

## 11. Summary & Best Practices

### OLAP Operations Covered:

| Operation | Description | Use Case |
|-----------|-------------|----------|
| **Slice** | Filter on one dimension | Q1 2024 transactions only |
| **Dice** | Filter on multiple dimensions | High-value international wires |
| **Drill-Down** | Navigate summary → detail | Currency → Type → Account |
| **Roll-Up** | Aggregate detail → summary | Daily → Weekly → Monthly |
| **Pivot** | Rotate data perspective | Type × Currency matrix |

### Complex Scenarios Demonstrated:

1. **Multi-Jurisdictional Sanctions Network** - Cross-border entity screening
2. **Fraud Ring Detection** - Coordinated suspicious activity
3. **Cross-Border Money Laundering** - Three-stage AML detection
4. **Advanced Vector Search** - Hybrid ML + business rules

### Best Practices:

1. **Use Aggregations** - Avoid fetching raw documents for analytics
2. **Filter Early** - Apply filters before aggregations for performance
3. **Limit Result Size** - Use `size: 0` when only aggregations needed
4. **Combine Approaches** - Hybrid queries (vector + rules) for accuracy
5. **Monitor Performance** - Profile queries for optimization

In [ ]:
# Cleanup
print("\n✅ Notebook Complete!")
print("\nTopics Covered:")
print("  ✓ Direct OpenSearch API calls")
print("  ✓ OLAP operations (Slice, Dice, Drill-Down, Roll-Up, Pivot)")
print("  ✓ Complex fraud and AML scenarios")
print("  ✓ Advanced vector search with business logic")
print("\nNext Steps:")
print("  1. Adapt queries to your data schema")
print("  2. Build automated detection pipelines")
print("  3. Integrate with alerting systems")
print("  4. Create dashboards from aggregation results")